In [1]:
import pandas as pd
from glob import glob
from time import strftime, sleep
import numpy as np
from datetime import datetime
from pandas_datareader import data as pdr
from pandas.tseries.offsets import BDay
import yfinance as yf
yf.pdr_override()

In [2]:
# simple function to make headers nicer
def clean_header(df):
    df.columns = df.columns.str.strip().str.lower().str.replace('.', '').str.replace('(', '').str.replace(')', '').str.replace(' ', '_').str.replace('_/_', '/')

# timestamp for file names
def get_now():
    now = datetime.now().strftime('%Y-%m-%d_%Hh%Mm')
    return now

In [34]:
all_transactions = pd.read_excel('df.xlsx')


In [35]:
all_tickers = list(all_transactions['ticker'].unique())
# some tickers may have been delisted. need to blacklist them here
blacklist = ['VSLR', 'HTZ']
filt_tickers = [tick for tick in all_tickers if tick not in blacklist]
print('You traded {} different stocks'.format(len(all_tickers)))

You traded 41 different stocks


In [36]:
# all transactions without the delisted stocks
final_filtered = all_transactions[~all_transactions.ticker.isin(blacklist)]

## Collecting the price history for all tickers

In [37]:
ly = datetime.today().year-1
today = datetime.today()
start_sp = datetime(2019, 1, 1)
end_sp = today
start_stocks = datetime(2019, 1, 1)
end_stocks = today
start_ytd = datetime(ly, 12, 31) + BDay(1)

def get(tickers, startdate, enddate):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['ticker', 'date']))

all_data = get(filt_tickers, start_stocks, end_stocks)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['FB']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['VIAC']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['TWTR']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['XLNX']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [38]:
clean_header(all_data)

<ipython-input-2-93d2dea9328a>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace('.', '').str.replace('(', '').str.replace(')', '').str.replace(' ', '_').str.replace('_/_', '/')
<ipython-input-2-93d2dea9328a>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace('.', '').str.replace('(', '').str.replace(')', '').str.replace(' ', '_').str.replace('_/_', '/')
<ipython-input-2-93d2dea9328a>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal

In [41]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 43393 entries, ('MMM', Timestamp('2019-01-02 00:00:00')) to ('GILT', Timestamp('2024-01-12 00:00:00'))
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   open       43393 non-null  float64
 1   high       43393 non-null  float64
 2   low        43393 non-null  float64
 3   close      43393 non-null  float64
 4   adj_close  43393 non-null  float64
 5   volume     43393 non-null  float64
dtypes: float64(6)
memory usage: 2.2+ MB


##### MEGA_DICT = dictionary with all the tickers as keys, and their ticker prices as DF
##### MEGA_DF = all the DF's from DICT, concatenated along the columns. Can use filter to select columns

In [42]:
MEGA_DICT = {}  # you have to create it first
min_date = '2020-01-01'  # optional
TX_COLUMNS = ['date','ticker', 'cashflow', 'cml_units', 'cml_cost', 'gain_loss']
tx_filt = all_transactions[TX_COLUMNS]  # keeping just the most relevant ones for now

for ticker in filt_tickers:
    prices_df = all_data[all_data.index.get_level_values('ticker').isin([ticker])].reset_index()
    ## Can add more columns like volume!
    PX_COLS = ['date', 'adj_close']
    prices_df = prices_df[prices_df.date >= min_date][PX_COLS].set_index(['date'])
    # Making sure we get sameday transactions
    tx_df = tx_filt[tx_filt.ticker==ticker].groupby('date').agg({'cashflow': 'sum',
                                                                 'cml_units': 'last',
                                                                 'cml_cost': 'last',
                                                                 'gain_loss': 'sum'})
    # Merging price history and transactions dataframe
    tx_and_prices = pd.merge(prices_df, tx_df, how='outer', left_index=True, right_index=True).fillna(0)
    # This is to fill the days that were not in our transaction dataframe
    tx_and_prices['cml_units'] = tx_and_prices['cml_units'].replace(to_replace=0, method='ffill')
    tx_and_prices['cml_cost'] = tx_and_prices['cml_cost'].replace(to_replace=0, method='ffill')
    tx_and_prices['gain_loss'] = tx_and_prices['gain_loss'].replace(to_replace=0, method='ffill')
    # Cumulative sum for the cashflow
    tx_and_prices['cashflow'] = tx_and_prices['cashflow'].cumsum()
    tx_and_prices['avg_price'] = (tx_and_prices['cml_cost']/tx_and_prices['cml_units'])
    tx_and_prices['mktvalue'] = (tx_and_prices['cml_units']*tx_and_prices['adj_close'])
    tx_and_prices = tx_and_prices.add_prefix(ticker+'_')
    # Once we're happy with the dataframe, add it to the dictionary
    MEGA_DICT[ticker] = tx_and_prices.round(3)

In [43]:
MEGA_DICT['RUN'].tail()

,RUN_adj_close,RUN_cashflow,RUN_cml_units,RUN_cml_cost,RUN_gain_loss,RUN_avg_price,RUN_mktvalue
date,,,,,,,
2024-01-08,16.64,201.08,10.0,336.386,429.891,33.639,166.4
2024-01-09,16.30,201.08,10.0,336.386,429.891,33.639,163.0
2024-01-10,16.31,201.08,10.0,336.386,429.891,33.639,163.1
2024-01-11,15.77,201.08,10.0,336.386,429.891,33.639,157.7
2024-01-12,15.10,201.08,10.0,336.386,429.891,33.639,151.0


In [46]:
MEGA_DF = pd.concat(MEGA_DICT.values(), axis=1)


## Portfolio DF

In [48]:
portf_allvalues = MEGA_DF.filter(regex='mktvalue').fillna(0) #  getting just the market value of each ticker
portf_allvalues['portf_value'] = portf_allvalues.sum(axis=1) # summing all market values
portf_allvalues['portf_value']

date
2020-01-02        0.000
2020-01-03        0.000
2020-01-06        0.000
2020-01-07        0.000
2020-01-08      316.738
                ...    
2024-01-08    14046.884
2024-01-09    13901.601
2024-01-10    13917.226
2024-01-11    13833.485
2024-01-12    13777.916
Name: portf_value, Length: 1015, dtype: float64

In [49]:
# For the S&P500 price return
sp500 = pdr.get_data_yahoo('^GSPC', start_stocks, end_sp)
clean_header(sp500)

[*********************100%%**********************]  1 of 1 completed


<ipython-input-2-93d2dea9328a>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace('.', '').str.replace('(', '').str.replace(')', '').str.replace(' ', '_').str.replace('_/_', '/')
<ipython-input-2-93d2dea9328a>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace('.', '').str.replace('(', '').str.replace(')', '').str.replace(' ', '_').str.replace('_/_', '/')
<ipython-input-2-93d2dea9328a>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal

In [50]:
#getting the pct change
portf_allvalues = portf_allvalues.join(sp500['adj_close'], how='inner')
portf_allvalues.rename(columns={'adj_close': 'sp500_mktvalue'}, inplace=True)
portf_allvalues['ptf_value_pctch'] = (portf_allvalues['portf_value'].pct_change()*100).round(2)
portf_allvalues['sp500_pctch'] = (portf_allvalues['sp500_mktvalue'].pct_change()*100).round(2)
portf_allvalues['ptf_value_diff'] = (portf_allvalues['portf_value'].diff()).round(2)
portf_allvalues['sp500_diff'] = (portf_allvalues['sp500_mktvalue'].diff()).round(2)

In [51]:
portf_allvalues.head()

,MMM_mktvalue,TSLA_mktvalue,AAPL_mktvalue,RUN_mktvalue,ZM_mktvalue,DIS_mktvalue,IBM_mktvalue,AMD_mktvalue,FB_mktvalue,BA_mktvalue,...,PLTR_mktvalue,PLL_mktvalue,TLRY_mktvalue,GILT_mktvalue,portf_value,sp500_mktvalue,ptf_value_pctch,sp500_pctch,ptf_value_diff,sp500_diff
2020-01-02,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,3257.850098,NaN,NaN,NaN,NaN
2020-01-03,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,3234.850098,NaN,-0.71,0.00,-23.00
2020-01-06,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,3246.280029,NaN,0.35,0.00,11.43
2020-01-07,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,3237.179932,NaN,-0.28,0.00,-9.10
2020-01-08,152.691,164.047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,316.738,3253.050049,inf,0.49,316.74,15.87


In [52]:
# KPI's for portfolio
kpi_portfolio7d_abs = portf_allvalues.tail(7).ptf_value_diff.sum().round(2)
kpi_portfolio15d_abs = portf_allvalues.tail(15).ptf_value_diff.sum().round(2)
kpi_portfolio30d_abs = portf_allvalues.tail(30).ptf_value_diff.sum().round(2)
kpi_portfolio200d_abs = portf_allvalues.tail(200).ptf_value_diff.sum().round(2)
kpi_portfolio7d_pct = (kpi_portfolio7d_abs/portf_allvalues.tail(7).portf_value[0]).round(3)*100
kpi_portfolio15d_pct = (kpi_portfolio15d_abs/portf_allvalues.tail(15).portf_value[0]).round(3)*100
kpi_portfolio30d_pct = (kpi_portfolio30d_abs/portf_allvalues.tail(30).portf_value[0]).round(3)*100
kpi_portfolio200d_pct = (kpi_portfolio200d_abs/portf_allvalues.tail(200).portf_value[0]).round(3)*100

In [53]:
# KPI's for S&P500
kpi_sp500_7d_abs = portf_allvalues.tail(7).sp500_diff.sum().round(2)
kpi_sp500_15d_abs = portf_allvalues.tail(15).sp500_diff.sum().round(2)
kpi_sp500_30d_abs = portf_allvalues.tail(30).sp500_diff.sum().round(2)
kpi_sp500_200d_abs = portf_allvalues.tail(200).sp500_diff.sum().round(2)
kpi_sp500_7d_pct = (kpi_sp500_7d_abs/portf_allvalues.tail(7).sp500_mktvalue[0]).round(3)*100
kpi_sp500_15d_pct = (kpi_sp500_15d_abs/portf_allvalues.tail(15).sp500_mktvalue[0]).round(3)*100
kpi_sp500_30d_pct = (kpi_sp500_30d_abs/portf_allvalues.tail(30).sp500_mktvalue[0]).round(3)*100
kpi_sp500_200d_pct = (kpi_sp500_200d_abs/portf_allvalues.tail(200).sp500_mktvalue[0]).round(3)*100

In [54]:
pip install dash

In [55]:
pip install dash_bootstrap_components

In [56]:
pip install jupyter_dash

In [57]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import plotly.express as px
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import dash_table
from jupyter_dash import JupyterDash

In [58]:
initial_date = '2020-05-30'  # do not use anything earlier than your first trade
plotlydf_portfval = portf_allvalues[portf_allvalues.index > initial_date]
plotlydf_portfval = plotlydf_portfval[['portf_value', 'sp500_mktvalue', 'ptf_value_pctch',
                                     'sp500_pctch', 'ptf_value_diff', 'sp500_diff']].reset_index().round(2)
# calculating cumulative growth since initial date
plotlydf_portfval['ptf_growth'] = plotlydf_portfval.portf_value/plotlydf_portfval['portf_value'].iloc[0]
plotlydf_portfval['sp500_growth'] = plotlydf_portfval.sp500_mktvalue/plotlydf_portfval['sp500_mktvalue'].iloc[0]
plotlydf_portfval.rename(columns={'index': 'date'}, inplace=True)  # needed for later
plotlydf_portfval

,date,portf_value,sp500_mktvalue,ptf_value_pctch,sp500_pctch,ptf_value_diff,sp500_diff,ptf_growth,sp500_growth
0,2020-06-01,8018.60,3055.73,10.83,0.38,783.64,11.42,1.000000,1.000000
1,2020-06-02,8140.48,3080.82,1.52,0.82,121.88,25.09,1.015200,1.008211
2,2020-06-03,8364.66,3122.87,2.75,1.36,224.18,42.05,1.043157,1.021972
3,2020-06-04,8153.74,3112.35,-2.52,-0.34,-210.92,-10.52,1.016853,1.018529
4,2020-06-05,8610.94,3193.93,5.61,2.62,457.20,81.58,1.073871,1.045227
...,...,...,...,...,...,...,...,...,...
907,2024-01-08,14046.88,4763.54,0.76,1.41,106.45,66.30,1.751787,1.558888
908,2024-01-09,13901.60,4756.50,-1.03,-0.15,-145.28,-7.04,1.733669,1.556584
909,2024-01-10,13917.23,4783.45,0.11,0.57,15.62,26.95,1.735618,1.565403
910,2024-01-11,13833.49,4780.24,-0.60,-0.07,-83.74,-3.21,1.725175,1.564353


In [59]:
CHART_THEME = 'plotly_white'  # others include seaborn, ggplot2, plotly_dark

chart_ptfvalue = go.Figure()  # generating a figure that will be updated in the following lines
chart_ptfvalue.add_trace(go.Scatter(x=plotlydf_portfval.date, y=plotlydf_portfval.portf_value,
                    mode='lines',  # you can also use "lines+markers", or just "markers"
                    name='Global Value'))
chart_ptfvalue.layout.template = CHART_THEME
chart_ptfvalue.layout.height=500
chart_ptfvalue.update_layout(margin = dict(t=50, b=50, l=25, r=25))  # this will help you optimize the chart space
chart_ptfvalue.update_layout(
#     title='Global Portfolio Value (USD $)',
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='Value: $ USD',
        titlefont_size=14,
        tickfont_size=12,
        ))
# chart_ptfvalue.update_xaxes(rangeslider_visible=False)
# chart_ptfvalue.update_layout(showlegend=False)
chart_ptfvalue.show()

In [60]:
import plotly.io as pio
list(pio.templates)  # doctest: +ELLIPSIS

['ggplot2',
 'seaborn',
 'simple_white',
 'plotly',
 'plotly_white',
 'plotly_dark',
 'presentation',
 'xgridoff',
 'ygridoff',
 'gridon',
 'none']

In [61]:
plotlydf_portfval

,date,portf_value,sp500_mktvalue,ptf_value_pctch,sp500_pctch,ptf_value_diff,sp500_diff,ptf_growth,sp500_growth
0,2020-06-01,8018.60,3055.73,10.83,0.38,783.64,11.42,1.000000,1.000000
1,2020-06-02,8140.48,3080.82,1.52,0.82,121.88,25.09,1.015200,1.008211
2,2020-06-03,8364.66,3122.87,2.75,1.36,224.18,42.05,1.043157,1.021972
3,2020-06-04,8153.74,3112.35,-2.52,-0.34,-210.92,-10.52,1.016853,1.018529
4,2020-06-05,8610.94,3193.93,5.61,2.62,457.20,81.58,1.073871,1.045227
...,...,...,...,...,...,...,...,...,...
907,2024-01-08,14046.88,4763.54,0.76,1.41,106.45,66.30,1.751787,1.558888
908,2024-01-09,13901.60,4756.50,-1.03,-0.15,-145.28,-7.04,1.733669,1.556584
909,2024-01-10,13917.23,4783.45,0.11,0.57,15.62,26.95,1.735618,1.565403
910,2024-01-11,13833.49,4780.24,-0.60,-0.07,-83.74,-3.21,1.725175,1.564353


In [62]:
fig2 = go.Figure(data=[
    go.Bar(name='Portfolio', x=plotlydf_portfval['date'], y=plotlydf_portfval['ptf_value_pctch']),
    go.Bar(name='SP500', x=plotlydf_portfval['date'], y=plotlydf_portfval['sp500_pctch'])
])
# Change the bar mode
fig2.update_layout(barmode='group')
fig2.layout.template = CHART_THEME
fig2.layout.height=300
fig2.update_layout(margin = dict(t=50, b=50, l=25, r=25))
fig2.update_layout(
#     title='% variation - Portfolio vs SP500',
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='% change',
        titlefont_size=14,
        tickfont_size=12,
        ))
fig2.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99))

fig2.show()

In [63]:
df = plotlydf_portfval[['date', 'ptf_growth', 'sp500_growth']].copy().round(3)
df['month'] = df.date.dt.month_name()  # date column should be formatted as datetime
df['weekday'] = df.date.dt.day_name()  # could be interesting to analyze weekday returns later
df['year'] = df.date.dt.year
df['weeknumber'] = df.date.dt.week    # could be interesting to try instead of timeperiod
df['timeperiod'] = df.year.astype(str) + ' - ' + df.date.dt.month.astype(str).str.zfill(2)
df.head(5)

<ipython-input-63-dc92a512f308>:5: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.



,date,ptf_growth,sp500_growth,month,weekday,year,weeknumber,timeperiod
0,2020-06-01,1.000,1.000,June,Monday,2020,23,2020 - 06
1,2020-06-02,1.015,1.008,June,Tuesday,2020,23,2020 - 06
2,2020-06-03,1.043,1.022,June,Wednesday,2020,23,2020 - 06
3,2020-06-04,1.017,1.019,June,Thursday,2020,23,2020 - 06
4,2020-06-05,1.074,1.045,June,Friday,2020,23,2020 - 06


In [64]:
# getting the percentage change for each period. the first period will be NaN
sp = df.reset_index().groupby('timeperiod').last()['sp500_growth'].pct_change()*100
ptf = df.reset_index().groupby('timeperiod').last()['ptf_growth'].pct_change()*100
plotlydf_growth_compare = pd.merge(ptf, sp, on='timeperiod').reset_index().round(3)
plotlydf_growth_compare.head()

,timeperiod,ptf_growth,sp500_growth
0,2020 - 06,NaN,NaN
1,2020 - 07,9.315,5.419
2,2020 - 08,38.099,7.009
3,2020 - 09,14.125,-3.843
4,2020 - 10,-6.122,-2.816


In [65]:
fig_growth2 = go.Figure()
fig_growth2.layout.template = CHART_THEME
fig_growth2.add_trace(go.Bar(
    x=plotlydf_growth_compare.timeperiod,
    y=plotlydf_growth_compare.ptf_growth.round(2),
    name='Portfolio'
))
fig_growth2.add_trace(go.Bar(
    x=plotlydf_growth_compare.timeperiod,
    y=plotlydf_growth_compare.sp500_growth.round(2),
    name='S&P 500',
))
fig_growth2.update_layout(barmode='group')
fig_growth2.layout.height=300
fig_growth2.update_layout(margin = dict(t=50, b=50, l=25, r=25))
fig_growth2.update_layout(
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='% change',
        titlefont_size=13,
        tickfont_size=12,
        ))

fig_growth2.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99))
fig_growth2.show()

In [66]:
indicators_ptf = go.Figure()
indicators_ptf.layout.template = CHART_THEME
indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio7d_pct,
    number = {'suffix': " %"},
    title = {"text": "<br><span style='font-size:0.7em;color:gray'>7 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_7d_pct, 'relative': False},
    domain = {'row': 0, 'column': 0}))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio15d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>15 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_15d_pct, 'relative': False},
    domain = {'row': 1, 'column': 0}))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio30d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>30 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_30d_pct, 'relative': False},
    domain = {'row': 2, 'column': 0}))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_portfolio200d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>200 Days</span>"},
    delta = {'position': "bottom", 'reference': kpi_sp500_200d_pct, 'relative': False},
    domain = {'row': 3, 'column': 1}))

indicators_ptf.update_layout(
    grid = {'rows': 4, 'columns': 1, 'pattern': "independent"},
    margin=dict(l=50, r=50, t=30, b=30)
)

In [67]:
indicators_sp500 = go.Figure()
indicators_sp500.layout.template = CHART_THEME
indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_7d_pct,
    number = {'suffix': " %"},
    title = {"text": "<br><span style='font-size:0.7em;color:gray'>7 Days</span>"},
    domain = {'row': 0, 'column': 0}))

indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_15d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>15 Days</span>"},
    domain = {'row': 1, 'column': 0}))

indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_30d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>30 Days</span>"},
    domain = {'row': 2, 'column': 0}))

indicators_sp500.add_trace(go.Indicator(
    mode = "number+delta",
    value = kpi_sp500_200d_pct,
    number = {'suffix': " %"},
    title = {"text": "<span style='font-size:0.7em;color:gray'>200 Days</span>"},
    domain = {'row': 3, 'column': 1}))

indicators_sp500.update_layout(
    grid = {'rows': 4, 'columns': 1, 'pattern': "independent"},
    margin=dict(l=50, r=50, t=30, b=30)
)

## Getting the prices for the TOP Stocks

In [68]:
last_positions = final_filtered.groupby(['ticker']).agg({'cml_units': 'last', 'cml_cost': 'last',
                                                'gain_loss': 'sum', 'cashflow': 'sum'}).reset_index()

In [80]:
stonk.info


{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '408 996 1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and pod

In [79]:
%%time
curr_prices = []
for tick in last_positions['ticker']:
    stonk = yf.Ticker(tick)
    price = stonk.info['regularMarketPrice']
    curr_prices.append(price)
    print(f'Done for {tick}')
len(curr_prices)

KeyError: 'regularMarketPrice'

In [76]:
last_positions['price'] = curr_prices
last_positions['current_value'] = (last_positions.price * last_positions.cml_units).round(2)
last_positions['avg_price'] = (last_positions.cml_cost / last_positions.cml_units).round(2)
last_positions = last_positions.sort_values(by='current_value', ascending=False)

TypeError: unsupported operand type(s) for *: 'dict' and 'float'

In [ ]:
last_positions

In [ ]:
donut_top = go.Figure()
donut_top.layout.template = CHART_THEME
donut_top.add_trace(go.Pie(labels=last_positions.head(15).ticker, values=last_positions.head(15).current_value))
donut_top.update_traces(hole=.7, hoverinfo="label+value+percent")
donut_top.update_traces(textposition='outside', textinfo='label+value')
donut_top.update_layout(showlegend=False)
donut_top.update_layout(margin = dict(t=50, b=50, l=25, r=25))
donut_top.show()

In [88]:
pip install dash


In [89]:
import dash

In [90]:
app = dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = dbc.Container(
    [
        dbc.Row(dbc.Col(html.H2('PORTFOLIO OVERVIEW', className='text-center text-primary, mb-3'))),  # header row

        dbc.Row([  # start of second row
            dbc.Col([  # first column on second row
            html.H5('Total Portfolio Value ($USD)', className='text-center'),
            dcc.Graph(id='chrt-portfolio-main',
                      figure=chart_ptfvalue,
                      style={'height':550}),
            html.Hr(),
            ], width={'size': 8, 'offset': 0, 'order': 1}),  # width first column on second row
            dbc.Col([  # second column on second row
            html.H5('Portfolio', className='text-center'),
            dcc.Graph(id='indicators-ptf',
                      figure=indicators_ptf,
                      style={'height':550}),
            html.Hr()
            ], width={'size': 2, 'offset': 0, 'order': 2}),  # width second column on second row
            dbc.Col([  # third column on second row
            html.H5('S&P500', className='text-center'),
            dcc.Graph(id='indicators-sp',
                      figure=indicators_sp500,
                      style={'height':550}),
            html.Hr()
            ], width={'size': 2, 'offset': 0, 'order': 3}),  # width third column on second row
        ]),  # end of second row

        dbc.Row([  # start of third row
            dbc.Col([  # first column on third row
                html.H5('Monthly Return (%)', className='text-center'),
                dcc.Graph(id='chrt-portfolio-secondary',
                      figure=fig_growth2,
                      style={'height':380}),
            ], width={'size': 8, 'offset': 0, 'order': 1}),  # width first column on second row
            dbc.Col([  # second column on third row
                html.H5('Top 15 Holdings', className='text-center'),
                dcc.Graph(id='pie-top15',
                      figure = donut_top,
                      style={'height':380}),
            ], width={'size': 4, 'offset': 0, 'order': 2}),  # width second column on second row
        ])  # end of third row

    ], fluid=True)

if __name__ == "__main__":
    app.run_server(debug=True, port=8058)

TypeError: 'module' object is not callable

In [82]:
app._terminate_server_for_port("localhost",8058)

AttributeError: 'JupyterDash' object has no attribute '_terminate_server_for_port'

In [ ]:
SIDEBAR_STYLE = {
    'position': 'fixed',
    'top': 0,
    'left': 0,
    'bottom': 0,
    'width': '12rem',
    'padding': '2rem 1rem',
    'background-color': 'lightgray',
}
CONTENT_STYLE = {
    'margin-left': '15rem',
    'margin-right': '2rem',
    'padding': '2rem' '1rem',
}

child = dbc.Container(
    [
        dbc.Row(dbc.Col(html.H2('PORTFOLIO OVERVIEW', className='text-center text-primary, mb-3'))),
        dbc.Row([
            dbc.Col([
            html.H5('Total Portfolio Value ($USD)', className='text-center'),
            dcc.Graph(id='chrt-portfolio-main',
                      figure=chart_ptfvalue,
                      style={'height':550}),
            html.Hr(),

            ],
                width={'size': 8, 'offset': 0, 'order': 1}),
            dbc.Col([
            html.H5('Portfolio', className='text-center'),
            dcc.Graph(id='indicators-ptf',
                      figure=indicators_ptf,
                      style={'height':550}),
            html.Hr()
            ],
                width={'size': 2, 'offset': 0, 'order': 2}),
            dbc.Col([
            html.H5('S&P500', className='text-center'),
            dcc.Graph(id='indicators-sp',
                      figure=indicators_sp500,
                      style={'height':550}),
            html.Hr()
            ],
                width={'size': 2, 'offset': 0, 'order': 3}),
        ]),  # end of second row
        dbc.Row([
            dbc.Col([
                html.H5('Monthly Return (%)', className='text-center'),
                dcc.Graph(id='chrt-portfolio-secondary',
                      figure=fig_growth2,
                      style={'height':380}),
            ],
                width={'size': 8, 'offset': 0, 'order': 1}),
            dbc.Col([
                html.H5('Top 15 Holdings', className='text-center'),
                dcc.Graph(id='pie-top15',
                      figure = donut_top,
                      style={'height':380}),
            ],
                width={'size': 4, 'offset': 0, 'order': 2}),
        ])

    ], fluid=True)

sidebar = html.Div(
    [
#         html.H5("Navigation Menu", className='display-6'),
        html.Hr(),
        html.P('Navigation Menu', className='text-center'),

        dbc.Nav(
            [
                dbc.NavLink('Home', href="/", active='exact'),
                dbc.NavLink('Page2', href="/page-2", active='exact')
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id='page-content', children=child, style=CONTENT_STYLE)

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SPACELAB])
# app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Location(id='url'),
    sidebar,
    content
])

if __name__ == "__main__":
    app.run_server(debug=True, port=8056)